# Discover the Higgs with Deep Neural Networks

The input data was created from 13 TeV ATLAS open data downloaded from http://opendata.atlas.cern/release/2020/documentation/index.html

For more information read:<br>
Review of the 13 TeV ATLAS Open Data release, Techn. Ber., All figures including auxiliary figures are available at https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PUBNOTES/ATL-OREACH-PUB-2020-001: CERN, 2020, url: http://cds.cern.ch/record/2707171

## Data Preparation

### Load the Data

In [ ]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import seed
import common

The goal of this lab course is to train a deep neural network to separate Higgs boson signal from background events. The most important signal sample ggH125_ZZ4lep corresponds to the process gg->H->ZZ. The dominant background sample is llll resulting from Z and ZZ decays.
After training the DNN model will be used to classify the events of the data samples.

Higgs signal samples:
- ggH125_ZZ4lep
- VBFH125_ZZ4lep
- WH125_ZZ4lep
- ZH125_ZZ4lep

Background samples:
- llll
- Zee
- Zmumu
- ttbar_lep

Data samples:
- data_A
- data_B
- data_C
- data_D

In [ ]:
# Define the input samples
sample_list_signal = ['ggH125_ZZ4lep']
sample_list_background = ['llll']

In [ ]:
# Read all the samples
data_frames = {}
for sample in sample_list_signal + sample_list_background:
    data_frames[sample] = pd.read_csv('input/' + sample + ".csv")

### Input Variables

The input provides several variables to classify the events. Since each event has multiple leptons, they were ordered in descending order based on their transverse momentum. Thus, lepton 1 has the highest transverse momentum, lepton 2 the second highest, and so on. <br>
Most of the given variables can be called low-level, because they represent event or object properties, which can be derived directly from the reconstruction in the detector. In contrast to this are high-level variables, which result from the combination of several low-level variables. In the given dataset the only high-level variables are invariant masses of multiple particles:<br>
$m_{inv} = \sqrt{\left(\sum\limits_{i=1}^{n} E_i\right)^2 - \left(\sum\limits_{i=1}^{n} \vec{p}_i\right)^2}$


List of all available variables:<br>
- Event number
 - Each simulated event has its own specific number. (not used for training)
 - Variable name: eventNumber
- Event weight
 - To combine simulated events and finally compare them to data each event has to be scaled by its respective event weight. (not used for training)
 - Variable name: weight
- Number of jets
 - Jets are particle showers which result primarily from quarks and gluons
 - Variable name: jet_n
- Invariant four lepton mass
 - The invariant mass $m_{inv}(l_1, l_2, l_3, l_4)$ of the four leptons is extremly sensitive to Higgs boson events. This variable is to be displayed later and thus not used for training
 - Variable name: lep_m_llll
- Invariant two lepton mass
 - Invariant masses $m_{inv}(l_i, l_j)$ of all combinations of two leptons
 - Variable names: lep_m_ll_12, lep_m_ll_13, lep_m_ll_14, lep_m_ll_23, lep_m_ll_24, lep_m_ll_34
- Transverse momentum of the leptons
 - The momentum in the plane transverse to the beam axis
 - Variable names: lep1_pt, lep2_pt, lep3_pt, lep4_pt
- Lepton azimuthal angle
 - The azimuthal angle $\phi$ is measured in the plane transverse to the beam axis
 - Variable name: lep1_phi, lep2_phi, lep3_phi, lep4_phi
- Lepton pseudo rapidity
 - The angle $\theta$ is measured between the lepton track and the beam axis. Since this angle is not invariant against boosts along the beam axis, the pseudo rapidity $\eta = - \ln{\tan{\frac{\theta}{2}}}$ is primarily used in the ATLAS analyses
 - Variable names: lep1_eta, lep2_eta, lep3_eta, lep4_eta
- Lepton energy
 - The energy of the leptons reconstructed from the calorimeter entries
 - Variable name:lep1_e, lep2_e, lep3_e, lep4_e
- Lepton PDG-ID
 - The lepton type is classified by a n umber given by the Particle-Data-Group. The lepton types are $pdg-id(e)=11$, $pdg-id(\mu)=13$ and $pdg-id(\tau)=15$
 - Variable name: lep1_pdgId, lep2_pdgId, lep3_pdgId, lep4_pdgId
- Lepton charge
 - The charge of the given lepton reconstructed by the lepton track
 - Variable name: lep1_charge, lep2_charge, lep3_charge, lep4_charge

### Event Pre-Selection

Although the final selection of the data is to be performed on the basis of a DNN, a rough pre-selection of the data is still useful.
For this purpose, selection criteria are defined, which return either true or false based on the event kinematics and thus decide whether the respective event is kept or discarded.
Suitable criteria for this analysis are very basic selections that must be clearly fulfilled by H->ZZ->llll processes.


Hint: What lepton types and charges are expected in the final state?

In [ ]:
def cut_lep_type(lep_type_0, lep_type_1, lep_type_2, lep_type_3):
    # Only keep events like eeee, mumumumu or eemumu
    sum_lep_type = lep_type_0 + lep_type_1 + lep_type_2 + lep_type_3
    return sum_lep_type == 44 or sum_lep_type == 48 or sum_lep_type == 52


def cut_lep_charge(lep_charge_0, lep_charge_1, lep_charge_2, lep_charge_3):
    # Only keep events where the sum of all lepton charges is zero
    sum_lep_charge = lep_charge_0 + lep_charge_1 + lep_charge_2 + lep_charge_3
    return sum_lep_charge == 0

In [ ]:
# Apply the chosen selection criteria
for sample in sample_list_signal + sample_list_background:
    # Cut on lepton type
    data_frames[sample] = data_frames[sample][np.vectorize(cut_lep_type)(
        data_frames[sample].lep1_pdgId,
        data_frames[sample].lep2_pdgId,
        data_frames[sample].lep3_pdgId,
        data_frames[sample].lep4_pdgId)]

    # Cut on lepton charge
    data_frames[sample] = data_frames[sample][np.vectorize(cut_lep_charge)(
        data_frames[sample].lep1_charge,
        data_frames[sample].lep2_charge,
        data_frames[sample].lep3_charge,
        data_frames[sample].lep4_charge)]

### Data Investigation

Before one can decide which variables are suitable for training, one must first get a feel for the input variables.
For this purpose, the input samples are merged into a set of signal events and a set of background events. Afterwards, the behavior of signal and background can be studied in multiple variables.

In [ ]:
# Merge the signal and background data frames
def merge_data_frames(sample_list, data_frames_dic):
    for sample in sample_list:
        if sample == sample_list[0]:
            output_data_frame = data_frames_dic[sample]
        else:
            output_data_frame = pd.contact(output_data_frame, data_frames_dic[sample])
    return output_data_frame

data_frame_signal = merge_data_frames(sample_list_signal, data_frames)
data_frame_background = merge_data_frames(sample_list_background, data_frames)

The function common.plot_hist(variable, data_frame_1, data_frame_2) plots the given variable of the two data sets.
The variable must be a dictionary containing atleast the variable to plot. Additionally one can also specify the binning (list or numpy array) and the xlabel. The created lots are automatically saved in the plots directory<br>
An example for the transverse momnetum of the leading lepton is given below:

In [ ]:
# allow bins: None!!!!

var_lep1_pt = {'variable': 'lep1_pt',
               'binning': np.linspace(0, 200, 100),
               'xlabel': '$lep_{pt}$[1] [GeV]'}

common.plot_hist(var_lep1_pt, data_frame_signal, data_frame_background)

In [ ]:
# plotting of all distributions summed!!!!

In [ ]:
# This is not correct!!!!!
for sample in sample_list_signal + sample_list_background:
    print(sample, np.mean(data_frames[sample].weight), len(data_frames[sample].weight))

## Training

In [ ]:
# The training input variables
input_variable_list = ['lep1_pt', 'lep2_pt', 'lep3_pt', 'lep4_pt']
input_variable_list += ['lep1_charge', 'lep2_charge', 'lep3_charge', 'lep4_charge']
input_variable_list += ['lep1_pdgId', 'lep2_pdgId', 'lep3_pdgId', 'lep4_pdgId']
input_variable_list += ['lep1_phi', 'lep2_phi', 'lep3_phi', 'lep4_phi']
input_variable_list += ['lep1_eta', 'lep2_eta', 'lep3_eta', 'lep4_eta']
#input_variable_list = ['lep_m_ll_12', 'lep_m_ll_13', 'lep_m_ll_14', 'lep_m_ll_23', 'lep_m_ll_24', 'lep_m_ll_34']

In [ ]:
# Create the training input
input_values = []
input_classification = []
for sample in sample_list_signal + sample_list_background:
    # Classify signal and background (and skip if data)
    if sample in sample_list_signal:
        # 1 if signal
        input_classification.append(np.ones(len(data_frames[sample])))
    elif sample in sample_list_background:
        # 0 if background
        input_classification.append(np.zeros(len(data_frames[sample])))
    else:
        continue
    input_values.append(data_frames[sample][input_variable_list])

# Merge the input
input_values = np.concatenate(input_values)
input_classification = np.concatenate(input_classification)

In [ ]:
np.random.seed(seed=420)

def split_indices(df, train_frac, test_frac, val_frac):
    #assert train_frac + val_frac + test_frac <= 1
    random_values = np.random.rand(len(df))
    range1 = random_values <= train_frac
    range2 = random_values <= train_frac + test_frac
    range3 = random_values <= train_frac + test_frac + val_frac
    train_index = range1
    test_index = ~range1 & range2
    val_index = ~range2 & range3
    return train_index, test_index, val_index

train_index, test_index, val_index = split_indices(input_values, 0.8, 0.1, 0.1)

In [ ]:
train_input_values = input_values[train_index]
train_input_classification = input_classification[train_index]
test_input_values = input_values[test_index]
test_input_classification = input_classification[test_index]
val_input_values = input_values[val_index]
val_input_classification = input_classification[val_index]

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=[len(input_variable_list)]),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(30, activation='relu'),
  tf.keras.layers.Dense(10, activation='relu'),
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
predictions = model(train_input_values).numpy()

In [ ]:
#tf.nn.softmax(predictions).numpy()

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
#loss_fn(train_input_classification, predictions).numpy()

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(train_input_values, train_input_classification, epochs=7)

In [ ]:
model.evaluate(test_input_values,  test_input_classification, verbose=2)

In [ ]:
test_prediction = model.predict(test_input_values)
print(test_prediction)